# Завдання 1: Оптимізація обробки запитів до масиву за допомогою LRU-кешу

In [1]:
import random
import time
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key):
        if key not in self.cache:
            return None
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

    def invalidate(self, index):
        keys_to_remove = [key for key in self.cache if key[0] <= index <= key[1]]
        for key in keys_to_remove:
            del self.cache[key]

In [3]:
def range_sum_no_cache(array, L, R):
    return sum(array[L:R+1])

def update_no_cache(array, index, value):
    array[index] = value

In [4]:
def range_sum_with_cache(array, L, R, cache):
    cached = cache.get((L, R))
    if cached is not None:
        return cached
    result = sum(array[L:R+1])
    cache.put((L, R), result)
    return result

def update_with_cache(array, index, value, cache):
    array[index] = value
    cache.invalidate(index)

In [5]:
N = 100_000
Q = 50_000
array = [random.randint(1, 100) for _ in range(N)]
queries = []
for _ in range(Q):
    if random.random() < 0.7:
        L = random.randint(0, N - 2)
        R = random.randint(L, N - 1)
        queries.append(('Range', L, R))
    else:
        idx = random.randint(0, N - 1)
        val = random.randint(1, 100)
        queries.append(('Update', idx, val))

In [7]:
array_no_cache = array[:]
start = time.time()
for q in queries:
    if q[0] == 'Range':
        range_sum_no_cache(array_no_cache, q[1], q[2])
    else:
        update_no_cache(array_no_cache, q[1], q[2])
no_cache_time = time.time() - start

array_with_cache = array[:]
cache = LRUCache(1000)
start = time.time()
for q in queries:
    if q[0] == 'Range':
        range_sum_with_cache(array_with_cache, q[1], q[2], cache)
    else:
        update_with_cache(array_with_cache, q[1], q[2], cache)
cache_time = time.time() - start

print(f"Час виконання без кешу: {round(no_cache_time, 2)} с")
print(f"Час виконання з кешем: {round(cache_time, 2)} с")

Час виконання без кешу: 9.31 с
Час виконання з кешем: 8.42 с
